In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

from keras.utils import load_img, img_to_array

from sklearn.metrics import accuracy_score

import glob
from google.colab import drive
import os
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/data.csv')
data.head()

,Unnamed: 0,bmi,gender,is_training,name
0,0,34.207396,Male,1,img_0.bmp
1,1,26.453720,Male,1,img_1.bmp
2,2,34.967561,Female,1,img_2.bmp
3,3,22.044766,Female,1,img_3.bmp
4,4,37.758789,Female,1,img_4.bmp


### Without Pre-trained model

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255)
directory = '/content/drive/MyDrive/BMI/Images'
train_set = train_datagen.flow_from_dataframe(
                                  data,
                                  directory,
                                  x_col = 'name',
                                  y_col = 'bmi',
                                  target_size = (64, 64),
                                #   batch_size = 32,
                                  class_mode = 'raw')

print("Image shape:", train_set.image_shape)
# print("Number of classes:", train_set.num_classes)

Found 3962 validated image filenames.
Image shape: (64, 64, 3)


/usr/local/lib/python3.10/dist-packages/keras/preprocessing/image.py:1137: UserWarning: Found 244 invalid image filename(s) in x_col="name". These filename(s) will be ignored.
  warnings.warn(


In [ ]:
classifier = Sequential()
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu'))        #  input_shape=train_set.image_shape,
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(keras.layers.Flatten())
classifier.add(keras.layers.Dense(units = 128, activation = 'relu'))
# classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(1))
classifier.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['accuracy'])

In [ ]:
classifier.fit(train_set, epochs=10)

Epoch 1/10
124/124 [==============================] - 11s 77ms/step - loss: 146.1459 - accuracy: 0.0000e+00
Epoch 2/10
124/124 [==============================] - 8s 67ms/step - loss: 77.9136 - accuracy: 0.0000e+00
Epoch 3/10
124/124 [==============================] - 8s 68ms/step - loss: 74.4128 - accuracy: 0.0000e+00
Epoch 4/10
124/124 [==============================] - 9s 68ms/step - loss: 71.5384 - accuracy: 0.0000e+00
Epoch 5/10
124/124 [==============================] - 8s 68ms/step - loss: 72.8606 - accuracy: 0.0000e+00
Epoch 6/10
124/124 [==============================] - 8s 67ms/step - loss: 66.8024 - accuracy: 0.0000e+00
Epoch 7/10
124/124 [==============================] - 8s 68ms/step - loss: 67.7004 - accuracy: 0.0000e+00
Epoch 8/10
124/124 [==============================] - 8s 67ms/step - loss: 68.9232 - accuracy: 0.0000e+00
Epoch 9/10
124/124 [==============================] - 8s 68ms/step - loss: 67.7251 - accuracy: 0.0000e+00
Epoch 10/10
124/124 [=======================

In [ ]:
img = load_img('/content/drive/MyDrive/Test_Image/test_image_01.png', target_size = (64, 64))
# img = load_img('/content/drive/MyDrive/Test_Image/PassportPhoto.jpg', target_size = (64, 64))
# img = load_img('/content/drive/MyDrive/Test_Image/PP_1.jpg', target_size = (64, 64))
img = img_to_array(img)           # Convert the image to an array
new_image = img.reshape(1, 64, 64, 3)
new_image = new_image / 255.0  # Rescale the pixel values

In [ ]:
prediction = classifier.predict(new_image)
predicted_bmi = prediction[0][0]
predicted_bmi

1/1 [==============================] - 0s 105ms/step


30.721361

### With VGG16

In [ ]:
import os
import numpy as np
import pandas as pd
import keras
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator

# Define the parameters
img_width, img_height = 224, 224
train_data_dir = '/content/drive/MyDrive/BMI'
csv_file = '/content/drive/MyDrive/data.csv'
batch_size = 16
num_epochs = 10

# Load the BMI data from the CSV file
bmi_data = pd.read_csv(csv_file)
bmi_dict = dict(zip(bmi_data['name'], bmi_data['bmi']))

# Create a data generator for training
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')

# Load the VGG16 model without the top layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Add a global average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully connected layer with 1024 units and ReLU activation
x = Dense(1024, activation='relu')(x)

# Add an output layer with 3 units and softmax activation
predictions = Dense(3, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.n // batch_size,
    epochs=num_epochs)

# Save the trained model
model.save('/content/drive/MyDrive/bmi_model.h5')

# Load the trained model
model = keras.models.load_model('/content/drive/MyDrive/bmi_model.h5')

# Create a data generator for testing
test_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=1,
    class_mode='categorical',
    subset='validation')

# Use the model to predict the BMI for each test image
predicted_bmi = {}
for i in range(test_generator.samples):
    test_image, _ = test_generator.next()
    filename = os.path.basename(test_generator.filepaths[i])
    bmi = bmi_dict[filename]
    predicted_bmi[filename] = bmi * model.predict(test_image)[0][0]

# Print the predicted BMI for each test image
# for filename, bmi in predicted_bmi.items():
    # print(filename, bmi)

# img = load_img('/content/drive/MyDrive/Test_Image/test_image_01.png', target_size = (64, 64))
img = load_img('/content/drive/MyDrive/Test_Image/PassportPhoto.jpg', target_size = (64, 64))
# img = load_img('/content/drive/MyDrive/Test_Image/PP_1.jpg', target_size = (64, 64))
img = img_to_array(img)           # Convert the image to an array
new_image = img.reshape(1, 64, 64, 3)
new_image = new_image / 255.0  # Rescale the pixel values

prediction = classifier.predict(new_image)
predicted_bmi = prediction[0][0]
predicted_bmi

Found 3170 images belonging to 1 classes.
Epoch 1/10


<ipython-input-8-fdb80ff714bb>:57: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


198/198 [==============================] - 115s 574ms/step - loss: 3.3671 - accuracy: 0.8516
Epoch 2/10
198/198 [==============================] - 113s 569ms/step - loss: 3.3656 - accuracy: 0.8478
Epoch 3/10
198/198 [==============================] - 112s 565ms/step - loss: 3.4066 - accuracy: 0.6737
Epoch 4/10
198/198 [==============================] - 112s 565ms/step - loss: 3.8795 - accuracy: 0.5327
Epoch 5/10
198/198 [==============================] - 113s 569ms/step - loss: 10.7706 - accuracy: 0.3643
Epoch 6/10
198/198 [==============================] - 113s 568ms/step - loss: 10.5938 - accuracy: 0.3840
Epoch 7/10
198/198 [==============================] - 113s 569ms/step - loss: 18.7375 - accuracy: 0.3668
Epoch 8/10
198/198 [==============================] - 113s 572ms/step - loss: 20.3060 - accuracy: 0.3789
Epoch 9/10
198/198 [==============================] - 113s 572ms/step - loss: 32.4377 - accuracy: 0.3519
Epoch 10/10
198/198 [==============================] - 112s 567ms/step

26.707766

### VGG with SVR

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten
from tensorflow.keras.preprocessing import image
from sklearn.metrics import mean_squared_error

# Helper function to load and preprocess an image
def load_and_preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img / 255.0  # Normalize image pixels to [0, 1]
    return img
    
# Set the path to the folder containing training images
image_folder = '/content/drive/MyDrive/BMI/Images'

# Set the path to the CSV file containing BMI values
bmi_csv_file = '/content/drive/MyDrive/data.csv'

# Load VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = Flatten()(x)
model = Model(inputs=base_model.input, outputs=x)

# Read the CSV file into a DataFrame
df = pd.read_csv(bmi_csv_file)

# Filter out rows where the corresponding image file doesn't exist
df = df[df['name'].apply(lambda x: os.path.exists(os.path.join(image_folder, x)))]

# Load and preprocess images and corresponding BMI values
image_paths = [os.path.join(image_folder, filename) for filename in df['name']]
bmi_values = df['bmi'].astype(float).values

# Extract features from the fc6 layer
features = []
for image_path in image_paths:
    img = load_and_preprocess_image(image_path)
    feature = model.predict(img)
    features.append(feature.flatten())

# Convert features and BMI values to numpy arrays
X = np.array(features)
y = np.array(bmi_values)

# Train epsilon support vector regression (SVR) model
epsilon = 0.1  # Adjust the value of epsilon as needed
svr = SVR(epsilon=epsilon)
svr.fit(X, y)

# Calculate predicted BMI values for the training set
predicted_bmi_train = svr.predict(X)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y, predicted_bmi_train))
print("RMSE: ", rmse)

# Now you can use the trained SVR model for BMI regression
# For example, given a new image 'test_image.jpg', you can extract its features
# using the VGG16 model and predict its BMI using the SVR model as follows:
test_image_path = '/content/drive/MyDrive/Test_Image/PassportPhoto.jpg'
test_image = load_and_preprocess_image(test_image_path)
test_feature = model.predict(test_image)
predicted_bmi = svr.predict(test_feature)

print("BMI: " + str(predicted_bmi[0]))




1/1 [==============================] - 0s 65ms/step
RMSE:  7.517489654851519
1/1 [==============================] - 0s 73ms/step
BMI: 31.55851754285865


In [ ]:
# from sklearn.svm import SVR

# # Mount Google Drive
# drive.mount('/content/drive')

# # Define the parameters
# img_width, img_height = 224, 224
# train_data_dir = '/content/drive/MyDrive/BMI'
# csv_file = '/content/drive/MyDrive/data.csv'
# batch_size = 16
# num_epochs = 10

# # Load the BMI data from the CSV file
# bmi_data = pd.read_csv(csv_file)
# bmi_dict = dict(zip(bmi_data['name'], bmi_data['bmi']))

# # Create a data generator for training
# train_datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)

# train_generator = train_datagen.flow_from_directory(
#     train_data_dir,
#     target_size=(img_width, img_height),
#     batch_size=batch_size,
#     class_mode=None,
#     subset='training')
# # Get the file paths from the generator
# file_paths = train_generator.filepaths

# # Load the VGG16 model without the top layer
# base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# # Extract features from the fc6 layer
# features = base_model.predict(train_generator)

# # Flatten the features
# features_flatten = features.reshape((features.shape[0], -1))

# # Get the BMI values
# bmi_values = np.array([bmi_dict[os.path.basename(file_path)] for file_path in file_paths])

# # Create an instance of the SVR model
# svr_model = SVR(epsilon=0.1)

# # Fit the SVR model to the training data
# svr_model.fit(features_flatten, bmi_values)

# # Load the trained VGG16 model
# trained_model = keras.models.load_model('/content/drive/MyDrive/bmi_model.h5')

# # Create a data generator for testing
# test_generator = train_datagen.flow_from_directory(
#     train_data_dir,
#     target_size=(img_width, img_height),
#     batch_size=1,
#     class_mode=None,
#     subset='validation')

# # Use the trained VGG16 model to extract features from the fc6 layer
# test_features = base_model.predict_generator(test_generator)
# test_features_flatten = test_features.reshape((test_features.shape[0], -1))

# # Predict the BMI using the SVR model
# predicted_bmi = svr_model.predict(test_features_flatten)

# # Print the predicted BMI for each test image
# for i, file_path in enumerate(test_generator.filepaths):
#     filename = os.path.basename(file_path)
   
# # Load the true BMI values for the test images
# y_true = np.array([bmi_dict[os.path.basename(file_path)] for file_path in test_generator.filepaths])

# # Calculate the RMSE
# rmse = np.sqrt(mean_squared_error(y_true, predicted_bmi))

# # Print the RMSE and accuracy
# print("Root Mean Squared Error (RMSE):", rmse)

# # img = load_img('/content/drive/MyDrive/Test_Image/test_image_01.png', target_size = (64, 64))
# img = load_img('/content/drive/MyDrive/Test_Image/PassportPhoto.jpg', target_size = (64, 64))
# # img = load_img('/content/drive/MyDrive/Test_Image/PP_1.jpg', target_size = (64, 64))
# img = img_to_array(img)           # Convert the image to an array
# new_image = img.reshape(1, 64, 64, 3)
# new_image = new_image / 255.0  # Rescale the pixel values

# prediction = svr_model.predict(new_image)
# predicted_bmi = prediction[0][0]
# print("BMI: " + str(predicted_bmi))

### ResNet

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.svm import SVR
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.preprocessing import image
from sklearn.metrics import mean_squared_error

# Set the path to the folder containing training images
image_folder = '/content/drive/MyDrive/BMI/Images'

# Set the path to the CSV file containing BMI values
bmi_csv_file = '/content/drive/MyDrive/data.csv'

# Load FaceNet model
input_shape = (160, 160, 3)
facenet_model = InceptionResNetV2(include_top=False, weights='imagenet', input_shape=input_shape)
x = facenet_model.output
x = Flatten()(x)
model = Model(inputs=facenet_model.input, outputs=x)

# Read the CSV file into a DataFrame
df = pd.read_csv(bmi_csv_file)

# Filter out rows where the corresponding image file doesn't exist
df = df[df['name'].apply(lambda x: os.path.exists(os.path.join(image_folder, x)))]

# Load and preprocess images and corresponding BMI values
image_paths = [os.path.join(image_folder, filename) for filename in df['name']]
bmi_values = df['bmi'].astype(float).values

# Extract features using FaceNet
features = []
for image_path in image_paths:
    img = image.load_img(image_path, target_size=input_shape[:2])
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img / 255.0  # Normalize image
    feature = model.predict(img)
    features.append(feature.flatten())

# Convert features and BMI values to numpy arrays
X = np.array(features)
y = np.array(bmi_values)

# Train epsilon support vector regression (SVR) model
epsilon = 0.1  # Adjust the value of epsilon as needed
svr = SVR(epsilon=epsilon)
svr.fit(X, y)

# Calculate predicted BMI values for the training set
predicted_bmi_train = svr.predict(X)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y, predicted_bmi_train))
print("RMSE: ", rmse)

# Now you can use the trained SVR model for BMI regression
# For example, given a new image 'test_image.jpg', you can extract its features
# using the FaceNet model and predict its BMI using the SVR model as follows:
test_image_path = '/content/drive/MyDrive/Test_Image/PassportPhoto.jpg'
img = image.load_img(test_image_path, target_size=input_shape[:2])
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = img / 255.0  # Normalize image
test_feature = model.predict(img)
predicted_bmi = svr.predict(test_feature)

print("BMI: ", predicted_bmi[0])


1/1 [==============================] - 0s 82ms/step
RMSE:  7.392338746066566
1/1 [==============================] - 0s 82ms/step
BMI:  28.64143419115667


### FaceNet

In [ ]:
import os
import numpy as np
from sklearn.svm import SVR
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from PIL import Image
import pandas as pd

# Set the path to the folder containing training images
image_folder = '/content/drive/MyDrive/BMI/Images'

# Set the path to the CSV file containing BMI values
bmi_csv_file = '/content/drive/MyDrive/data.csv'

# Load VGG16 model
input_shape = (224, 224, 3)
vgg16_model = VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=input_shape))
x = vgg16_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)  # Additional fully connected layer
model = Model(inputs=vgg16_model.input, outputs=x)

# Load and preprocess images and corresponding BMI values
image_paths = []
bmi_values = []
df = pd.read_csv(bmi_csv_file)  # Read CSV as a DataFrame
for _, row in df.iterrows():
    image_path = os.path.join(image_folder, row['name'])
    if os.path.exists(image_path):
        image_paths.append(image_path)
        bmi_values.append(float(row['bmi']))

# Extract features using VGG16
features = []
for image_path in image_paths:
    img = image.load_img(image_path, target_size=input_shape[:2])
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    feature = model.predict(img)
    features.append(feature.flatten())

# Convert features and BMI values to numpy arrays
X = np.array(features)
y = np.array(bmi_values)

# Train epsilon support vector regression (SVR) model
epsilon = 0.1  # Adjust the value of epsilon as needed
svr = SVR(epsilon=epsilon)
svr.fit(X, y)

# Calculate predicted BMI values for the training set
predicted_bmi_train = svr.predict(X)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y, predicted_bmi_train))
print("RMSE: ", rmse)

# Now you can use the trained SVR model for BMI regression
# For example, given a new image 'test_image.jpg', you can extract its features
# using the VGG16 model and predict its BMI using the SVR model as follows:
test_image_path = '/content/drive/MyDrive/Test_Image/PassportPhoto.jpg'
img = image.load_img(test_image_path, target_size=input_shape[:2])
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = preprocess_input(img)
test_feature = model.predict(img)
predicted_bmi = svr.predict(test_feature)

print("BMI: ", predicted_bmi[0])


1/1 [==============================] - 0s 72ms/step
RMSE:  7.347312518049406
1/1 [==============================] - 0s 71ms/step
BMI:  31.5310676515658
